In [37]:
#required libraries
import numpy as np
import pygame
import tensorflow as tf
import time

In [38]:
# global constants

SCREEN_WIDTH, SCREEN_HEIGHT = 1280, 720
BG_COLOR = (120, 120, 120)
DRAW_COLOR = (50, 50, 50)

DRAW_RADIUS = 2
ERASE_RADIUS = 5

CAR_WIDTH, CAR_HEIGHT = 20, 40
DEFAULT_START_X, DEFAULT_START_Y = 950, 470
DEFAULT_START_ANGLE = 0
DEFAULT_START_SPEED = 0
ACCELERATION = 0.1
FRICTION = 0.05
TURN_ANGLE = 2

TRACK_SAVE_PATH = "track.png"
CAR_IMAGE_PATH = "Track_images/car.png"
TRACK_IMAGE_PATH = r"track.png"

FOR making th track drawing 
left ckick for drawing the track
right click for eraing the track

In [39]:
def draw_track():
    """
    Function to draw the racing track.
    Left mouse button to draw, right mouse button to erase.
    Press 'S' to save the track as an image.
    """
    pygame.init()
    pygame.display.set_caption("Drawing the track, press 'S' to save as image.")
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    screen.fill(BG_COLOR)
    running, drawing, erase = True, False, False
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    drawing = True
                elif event.button == 3:
                    erase = True
            elif event.type == pygame.MOUSEBUTTONUP:
                if event.button == 1:
                    drawing = False
                elif event.button == 3:
                    erase = False
            elif event.type == pygame.MOUSEMOTION:
                mouse_pos = pygame.mouse.get_pos()
                if drawing:
                    pygame.draw.circle(screen, DRAW_COLOR, mouse_pos, DRAW_RADIUS)
                elif erase:
                    pygame.draw.circle(screen, BG_COLOR, mouse_pos, ERASE_RADIUS)
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_s:
                pygame.image.save(screen, TRACK_SAVE_PATH)
        pygame.display.update()
    pygame.quit()

In [40]:
class Car:
    """
    Class representing a car in the racing simulation.
    """
    def __init__(self, image_path, x, y, angle=0, speed=0):
        image = pygame.image.load(image_path).convert_alpha()
        self.image = pygame.transform.scale(image, (CAR_WIDTH, CAR_HEIGHT))
        self.x, self.y = x, y
        self.angle = angle
        self.speed = speed

    def move(self):
        rad = np.radians(self.angle)
        self.x += self.speed * np.cos(rad)
        self.y -= self.speed * np.sin(rad)

    def draw(self, screen):
        rotated_img = pygame.transform.rotate(self.image, self.angle)
        rect = rotated_img.get_rect(center=(self.x, self.y))
        screen.blit(rotated_img, rect.topleft)

In [41]:
def ray_casting(car, screen, track_surface):
    sensor_distance = []
    sensor_endpoint = []
    sensor_angle = [-45, 0, 45]

    for angle in sensor_angle:
        ray_angle = car.angle + angle
        ray_x, ray_y = car.x, car.y
        distance = 0
        max_distance = 200

        while distance < max_distance:
            rad = np.radians(ray_angle)
            ray_x += np.cos(rad)
            ray_y -= np.sin(rad)
            distance += 1

            if not (0 <= ray_x < SCREEN_WIDTH and 0 <= ray_y < SCREEN_HEIGHT):
                break

            pixel_color = track_surface.get_at((int(ray_x), int(ray_y)))[0:3]
            if pixel_color == DRAW_COLOR:
                break

        sensor_distance.append(distance)
        sensor_endpoint.append((ray_x, ray_y))

    return sensor_distance, sensor_endpoint

In [42]:
def run_simulation(
    start_x=DEFAULT_START_X,
    start_y=DEFAULT_START_Y,
    start_angle=DEFAULT_START_ANGLE,
    start_speed=DEFAULT_START_SPEED
):
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    pygame.display.set_caption("Car Simulation")

    track_image = pygame.image.load(TRACK_IMAGE_PATH).convert()
    track_surface = track_image.copy()

    car = Car(CAR_IMAGE_PATH, start_x, start_y, start_angle, start_speed)

    screen.fill(BG_COLOR)
    screen.blit(track_image, (0, 0))

    CAR_ROAD_COLOR = track_surface.get_at((start_x, start_y))[:3]

    car_max_speed = 0
    car_best_lap_time = 0
    start_time = time.time()
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        time_running=True
        if not time_running:
            start_time = time.time()
        keys = pygame.key.get_pressed()
        if car.speed > 0:
            car.speed -= FRICTION
        if car.speed < 0:
            car.speed += FRICTION
        if abs(car.speed) < FRICTION:
            car.speed = 0
        if keys[pygame.K_DOWN]:
            car.speed -= ACCELERATION
        if keys[pygame.K_UP]:
            car.speed += ACCELERATION
        if car.speed != 0:
            turn_factor = TURN_ANGLE * (car.speed * 2 / 2.5)
            if keys[pygame.K_LEFT]:
                car.angle += turn_factor
            if keys[pygame.K_RIGHT]:
                car.angle -= turn_factor

        screen.blit(track_image, (0, 0))

        font = pygame.font.Font(None, 22)
        text_surface = font.render(f"Speed: {car.speed:.2f}", True, (255, 255, 255))
        screen.blit(text_surface, (50, 50))

        # Ray casting
        distance, sensor_endpoint = ray_casting(car, screen, track_surface)
        text_surface = font.render(f"Distances: {distance}", True, (255, 255, 255))
        screen.blit(text_surface, (50, 80))
        for endpoint in sensor_endpoint:
            pygame.draw.line(screen, (0, 255, 0), (car.x, car.y), endpoint, 1)

        car.move()
        car.draw(screen)

        # Collision detection (including out-of-bounds)
        crashed = False
        car_center_pos = (int(car.x), int(car.y))
        if not (0 <= car.x < SCREEN_WIDTH and 0 <= car.y < SCREEN_HEIGHT):
            crashed = True
        else:
            try:
                pixel_color = track_surface.get_at(car_center_pos)[:3]
                if pixel_color != CAR_ROAD_COLOR:
                    crashed = True
            except IndexError:
                crashed = True

        if crashed:
            end_time = time.time()
            elapsed_time = end_time - start_time
            if car_best_lap_time > elapsed_time or car_best_lap_time == 0:
                car_best_lap_time = elapsed_time
            text_surface = font.render(f"Current lap time : {elapsed_time:.2f}s", True, (255, 255, 255))
            screen.blit(text_surface, (50, 150))
            text_surface = font.render("Car has crashed!", True, (255, 255, 255))
            screen.blit(text_surface, (50, 110))
            pygame.display.update()
            pygame.time.wait(1000)  # Show message for 1 second
            car.x, car.y = start_x, start_y
            car.angle = start_angle
            car.speed = start_speed
            start_time = time.time()  # Reset lap timer after crash

        if car.speed > car_max_speed:
            car_max_speed = car.speed

        text_surface = font.render(f"Max speed: {car_max_speed:.2f}", True, (255, 255, 255))
        screen.blit(text_surface, (50, 500))
        text_surface = font.render(f"Best lap time: {car_best_lap_time:.2f}s", True, (255, 255, 255))
        screen.blit(text_surface, (50, 530))

        pygame.display.update()
        clock.tick(60)

    pygame.quit()

        if event.type == pygame.MOUSEMOTION:
            mouse_point = event.pos
            text_surface = font.render(f"Location: {mouse_point}", True, (255, 255, 255))
            screen.blit(text_surface, (50, 100))
 #code for getting mouse location
 


In [43]:
if __name__ == "__main__":
    #draw_track()
    run_simulation()
